In [86]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import missingno as msno
from sklearn.model_selection import train_test_split # For splitting data into training and testing sets
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder  # For converting categorical data to numerical values
from sklearn.metrics import accuracy_score  # For calculating the accuracy score of a classification model
from sklearn.impute import KNNImputer  # For imputing missing values using K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier  # K-Nearest Neighbors classification model
from xgboost import XGBClassifier  # XGBoost classification model
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import ADASYN
%matplotlib inline

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [87]:
# reading csv file 
df_train = pd.read_csv("/kaggle/input/titanic/train.csv")
df_test = pd.read_csv("/kaggle/input/titanic/test.csv")
Y = df_train["Survived"]
duplicate_values = df_train.duplicated()
#print(f'Duplicate values = {duplicate_values}')
passenger_ids = df_test['PassengerId'].copy()

remove_col = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked','Survived']
remove_col_val = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
x = df_train.drop(labels=remove_col, axis=1)
X_val = df_test.drop(labels=remove_col_val, axis=1)

print(x.describe(include = "all"))
df_test.head(20)
miss_values = df_test.isnull().sum()
print(miss_values)


            Pclass   Sex         Age       SibSp       Parch        Fare
count   891.000000   891  714.000000  891.000000  891.000000  891.000000
unique         NaN     2         NaN         NaN         NaN         NaN
top            NaN  male         NaN         NaN         NaN         NaN
freq           NaN   577         NaN         NaN         NaN         NaN
mean      2.308642   NaN   29.699118    0.523008    0.381594   32.204208
std       0.836071   NaN   14.526497    1.102743    0.806057   49.693429
min       1.000000   NaN    0.420000    0.000000    0.000000    0.000000
25%       2.000000   NaN   20.125000    0.000000    0.000000    7.910400
50%       3.000000   NaN   28.000000    0.000000    0.000000   14.454200
75%       3.000000   NaN   38.000000    1.000000    0.000000   31.000000
max       3.000000   NaN   80.000000    8.000000    6.000000  512.329200
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch     

In [88]:
x['FamilySize'] = x['SibSp'] + x['Parch'] + 1  # Add 1 to include the passenger themselves
x['AgeGroup'] = pd.cut(x['Age'], bins=[0, 12, 18, 35, 60, 100], labels=['Child', 'Teenager', 'Adult', 'Middle-aged', 'Senior'])

x.head()

X_val['FamilySize'] = X_val['SibSp'] + X_val['Parch'] + 1  # Add 1 to include the passenger themselves
X_val['AgeGroup'] = pd.cut(X_val['Age'], bins=[0, 12, 18, 35, 60, 100], labels=['Child', 'Teenager', 'Adult', 'Middle-aged', 'Senior'])

X_val.head()

print(f'len of passenger_id = {len(passenger_ids)}')
print(f'len of predicted_classes = {len(X_val)}')


len of passenger_id = 418
len of predicted_classes = 418


In [89]:
# Converting categorical values in 'Sex' to labels.
le = LabelEncoder()
x['Sex'] = le.fit_transform(x['Sex'])
x['AgeGroup'] = le.fit_transform(x['AgeGroup'])

X_val['Sex'] = le.fit_transform(X_val['Sex'])
X_val['AgeGroup'] = le.fit_transform(X_val['AgeGroup'])


print(x.head())

# Assuming original column names before imputation
columns = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare','FamilySize', 'AgeGroup']

# Impute missing values
imputer = KNNImputer(n_neighbors=5)
x_df = imputer.fit_transform(x)
X_val = imputer.fit_transform(X_val)
# Convert the NumPy array back to a DataFrame with original column names
x_df = pd.DataFrame(x_df, columns=columns)
X_val = pd.DataFrame(X_val, columns=columns)

x_df.head()
X_val.head()
print(len(X_val))


   Pclass  Sex   Age  SibSp  Parch     Fare  FamilySize  AgeGroup
0       3    1  22.0      1      0   7.2500           2         0
1       1    0  38.0      1      0  71.2833           2         2
2       3    0  26.0      0      0   7.9250           1         0
3       1    0  35.0      1      0  53.1000           2         0
4       3    1  35.0      0      0   8.0500           1         0
418


In [97]:
sc = StandardScaler()
x_df = sc.fit_transform(x_df)
X_val = sc.fit_transform(X_val)

# Convert the NumPy array back to a DataFrame with original column names
x_df = pd.DataFrame(x_df, columns=columns)
X_val = pd.DataFrame(X_val, columns=columns)
print(len(X_val))
X_val.head()

418


,Pclass,Sex,Age,SibSp,Parch,Fare,FamilySize,AgeGroup
0,0.873482,0.755929,0.379751,-0.499470,-0.400248,-0.497142,-0.553443,-0.964429
1,0.873482,-1.322876,1.291600,0.616992,-0.400248,-0.512006,0.105643,0.053579
2,-0.315819,0.755929,2.385818,-0.499470,-0.400248,-0.463833,-0.553443,0.562584
3,0.873482,0.755929,-0.167359,-0.499470,-0.400248,-0.482206,-0.553443,-0.964429
4,0.873482,-1.322876,-0.532098,0.616992,0.619896,-0.417228,0.764728,-0.964429


In [91]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, TensorDataset

# Split the data into training + validation and test sets
X_train, X_test, y_train, y_test = train_test_split(x_df, Y, test_size=0.15, random_state=0)

# Further split the training data into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)  # 0.25 * 0.8 = 0.2

# Convert the splits to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Create PyTorch datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


In [104]:


def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Set the seed
set_seed(42)


class SimpleNN(nn.Module):
    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)    # Increase number of neurons
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, 1)
        
        self.leaky_relu = nn.LeakyReLU()
        self.dropout = nn.Dropout(0.5)
        self.batchnorm1 = nn.BatchNorm1d(256)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.leaky_relu(self.batchnorm1(self.fc1(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.batchnorm2(self.fc2(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.batchnorm3(self.fc3(x)))
        x = self.dropout(x)
        x = self.leaky_relu(self.fc4(x))
        x = self.dropout(x)
        x = self.leaky_relu(self.fc5(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc6(x))
        return x



In [123]:
# Initialize the model
input_size = X_train_tensor.shape[1]  # Number of features
model = SimpleNN(input_size)

# Define the loss function and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=0.01)




In [124]:
# Training the model
num_epochs = 250
model.train()
best_val_loss = float('inf')
best_model_weights = None


for epoch in range(num_epochs):
    model.train()  # Ensure model is in training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        running_loss += loss.item()
    
    # Calculate the average training loss for the epoch
    avg_train_loss = running_loss / len(train_loader)
    
    # Evaluate the model on the validation set
    model.eval()  # Ensure model is in evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(test_loader)
    
    # Save the model weights if the validation loss is the best we've seen so far
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        best_model_weights = model.state_dict()
    
    # Print the average losses for this epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}")


Epoch [1/250], Train Loss: 0.7102, Val Loss: 0.7064
Epoch [2/250], Train Loss: 0.7126, Val Loss: 0.7028
Epoch [3/250], Train Loss: 0.7159, Val Loss: 0.7014
Epoch [4/250], Train Loss: 0.7148, Val Loss: 0.6982
Epoch [5/250], Train Loss: 0.6948, Val Loss: 0.6960
Epoch [6/250], Train Loss: 0.6957, Val Loss: 0.6935
Epoch [7/250], Train Loss: 0.6949, Val Loss: 0.6921
Epoch [8/250], Train Loss: 0.6872, Val Loss: 0.6898
Epoch [9/250], Train Loss: 0.6927, Val Loss: 0.6879
Epoch [10/250], Train Loss: 0.6932, Val Loss: 0.6864
Epoch [11/250], Train Loss: 0.6860, Val Loss: 0.6836
Epoch [12/250], Train Loss: 0.6823, Val Loss: 0.6825
Epoch [13/250], Train Loss: 0.6839, Val Loss: 0.6797
Epoch [14/250], Train Loss: 0.6780, Val Loss: 0.6764
Epoch [15/250], Train Loss: 0.6775, Val Loss: 0.6736
Epoch [16/250], Train Loss: 0.6770, Val Loss: 0.6743
Epoch [17/250], Train Loss: 0.6730, Val Loss: 0.6711
Epoch [18/250], Train Loss: 0.6708, Val Loss: 0.6699
Epoch [19/250], Train Loss: 0.6800, Val Loss: 0.6690
Ep

In [126]:
model.load_state_dict(best_model_weights)

# Set the model to evaluation mode
model.eval()

# Evaluate the best model on the validation (or test) set
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:  # You can also use val_loader if you want to evaluate on the validation set
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Best Model Accuracy: {accuracy:.2f}%')

torch.save(best_model_weights, '/kaggle/working/best_model_weights.pth')

Best Model Accuracy: 82.09%


In [127]:
import pandas as pd
import torch


model.load_state_dict(torch.load('/kaggle/working/best_model_weights.pth'))

# Run inference
model.eval()
with torch.no_grad():
    predictions = model(X_val_tensor)
    predicted_classes = (predictions > 0.5).int().view(-1)  # Convert to binary classes
    
# Assuming X_val is already preprocessed and is a tensor ready for inference
print(f'len of passenger_id = {len(passenger_ids)}')
print(f'len of predicted_classes = {len(predicted_classes)}')

# Create a DataFrame with PassengerId and Survived
output_df = pd.DataFrame({
    'PassengerId': passenger_ids,  # Use the stored PassengerId
    'Survived': predicted_classes.numpy()
})

# Save to CSV
output_df.to_csv('/kaggle/working/submission.csv', index=False)


len of passenger_id = 418
len of predicted_classes = 418
